**This is the first part of the assignment**.

**Part-1: Import neighborhood data for Toronto from Wikipedia and clean up the data so that it can be used for further anaylsis**. 

In [45]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt

Libraries imported.


In [3]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

In [38]:
url_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
url_source = requests.get(url_link).text
soup = BeautifulSoup(url_source, 'xml')
table=soup.find('table')

In [39]:
#create a new dataframe that consists of postal code, borough and neighborhood columns
columnnames = ['Postalcode','Borough','Neighborhood']
Toronto_df = pd.DataFrame(columns = columnnames)

In [40]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        Toronto_df.loc[len(Toronto_df)] = row_data

Toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [41]:
#Process data such that entries that have Borough = Not Assigned are not considered
Toronto_df_updated = Toronto_df[Toronto_df.Borough != 'Not assigned']

In [42]:
#Combine Neighborhoods that have the same Postal Code
Toronto_df_clean = Toronto_df_updated.groupby(['Postalcode','Borough'], sort = False).agg(','.join)
Toronto_df_clean.reset_index(inplace=True)

In [43]:
#Replace Neighborhood fields that have "Not assigned" with the Borough name
Toronto_df_clean['Neighborhood'] = np.where(Toronto_df_clean['Neighborhood'] == 'Not assigned',Toronto_df_clean['Borough'], Toronto_df_clean['Neighborhood'])
Toronto_df_clean.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [44]:
#shape of the dataframe
Toronto_df_clean.shape

(103, 3)

**This is the second part of the assignment**.

**Part-2: Create a dataframe that consists the latitude and longitude information for all boroughs/neighborhoods in Toronto**.

In [54]:
#Import latitude and longitude data
lat_long = pd.read_csv('https://cocl.us/Geospatial_data')
lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [55]:
lat_long.rename(columns={'Postal Code': 'Postalcode'},inplace=True)
lat_long_merge = pd.merge(Toronto_df_clean,lat_long,on='Postalcode')
lat_long_merge.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


**This is the third part of the assignment**

**Part-3: Explore and cluster neighborhoods in Toronto**.

**Get rows that contain the string "Toronto" in the Borough field**

In [104]:
Toronto_Borough_data = lat_long_merge[lat_long_merge['Borough'].str.contains("Toronto")]
Toronto_Borough_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


**Get the latitude and longitude for Toronto to use with Folium map**

In [105]:

address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torono are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Torono are 43.6534817, -79.3839347.


**Use Folium to visualize neighborhoods in Toronto**

In [121]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough,neighborhood in zip(Toronto_Borough_data['Latitude'], Toronto_Borough_data['Longitude'],Toronto_Borough_data['Borough'], Toronto_Borough_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

**Explore the neighborhoods in Toronto**

*Define Foursqure IDs**

In [107]:
CLIENT_ID = 'BCRJAFBBWJX0FCUTENRD4VWGVJ4ZQKNNJL0AZQHYMYPR0JHI' # your Foursquare ID
CLIENT_SECRET = 'FXFUBQ3NMXFL0TW5QVTALFLHGRAOMJ2LDJM0F0JPHNDMQBAI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BCRJAFBBWJX0FCUTENRD4VWGVJ4ZQKNNJL0AZQHYMYPR0JHI
CLIENT_SECRET:FXFUBQ3NMXFL0TW5QVTALFLHGRAOMJ2LDJM0F0JPHNDMQBAI


**Find a neighborhood in dataframe and explore that neighborhood**

In [108]:
Toronto_Borough_data['Neighborhood'].iloc[6]

'Central Bay Street'

**Find the latitude and longitude of the above neighborhood**

In [109]:
neighborhood_latitude = Toronto_Borough_data['Latitude'].iloc[6] # neighborhood latitude value
neighborhood_longitude = Toronto_Borough_data['Longitude'].iloc[6] # neighborhood longitude value

neighborhood_name = Toronto_Borough_data['Neighborhood'].iloc[6] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


**Get the top 100 venues that are in the Central Bay Street within a radius of 500 meters**.

In [110]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

**Send GET request and view results**

In [111]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3a90b4c9c6b914fd5123f9'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 64,
  'suggestedBounds': {'ne': {'lat': 43.6624524045, 'lng': -79.38117421839567},
   'sw': {'lat': 43.6534523955, 'lng': -79.39359098160432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
        'label

In [112]:
#borrow the get_category_type
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

**Clean the json and structure into a dataframe**

In [113]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870


In [114]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

64 venues were returned by Foursquare.


**Cluster Neighborhoods - Use K-Means clustering**

In [115]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_Borough_data.drop(['Postalcode','Borough','Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 4, 0, 0, 3, 0, 1], dtype=int32)

In [116]:
# add clustering labels
Toronto_Borough_data.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_Borough_data.head()

,Cluster Labels,Postalcode,Borough,Neighborhood,Latitude,Longitude
2,0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,0,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,0,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,0,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,4,M4E,East Toronto,The Beaches,43.676357,-79.293031


**Create Folium map**

In [120]:
# create map
map_clusters = folium.Map(location=[neighborhood_latitude,neighborhood_longitude],zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(Toronto_Borough_data['Latitude'], Toronto_Borough_data['Longitude'], Toronto_Borough_data['Neighborhood'], Toronto_Borough_data['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters